# Assignment 6: Machine Learning Fashionista 2.0
## Ang Li-Lian

In [1]:
from glob import glob
from keras.preprocessing.image import load_img, img_to_array
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.layers import Dense, Flatten, Dropout
from keras.models import Model
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from PIL import Image

INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2


In [2]:
# Load all the images
women_files = glob(r'C:\Users\lilia\Desktop\CS156\04-1\women\*')
men_files = glob(r'C:\Users\lilia\Desktop\CS156\04-1\men\*')

imSize = (64,64)

flattened = []
# for each image path
for gender in [women_files,men_files]:
    for path in gender:
        # open it as a read file in binary mode
        with open(path, 'r+b') as f:
            # open it as an image
            with Image.open(f) as image:
                # resize the image to be more manageable
                img = image.resize(imSize)
                img = img_to_array(img)
                img = img.reshape((img.shape[0], img.shape[1], img.shape[2]))
                flattened.append(img)

label = [0]*len(women_files) +[1]*len(men_files)

In [3]:
# Split into training and test set
X_train, X_test, y_train, y_test = train_test_split(
    np.array(flattened), np.array(label), 
    test_size=0.2, stratify=np.array(label))

# Support Vector Machines (SVM)

There are two parameters we need to select for each SVM: C and gamma. C determines how smooth the function will be or how much it fits the data. The larger the value of C, the more closely the function attempts to fit the data. Gamma determines how much each data point's neighbours influences its classification. The larger the gamma, the smaller the radius of neighbours each point takes into account.

To determine the best paramteres for each SVM, we will perform cross-validation and choose the best parameters based on their accuracy.

In [4]:
# Reshape data for SVM
svm_train  = X_train.reshape(-1,64*64*3)
svm_test = X_test.reshape(-1,64*64*3)

In [5]:
def cv(kernel):
    accuracy, g, C = 0, 0,0
    for gamma in 10.0**np.arange(-5,2):
        for c in 10.0**np.arange(-5,2):
            if kernel == "poly":
                svm = SVC(kernel = kernel, gamma = gamma, C = c, degree = 2)
            else:
                svm = SVC(kernel = kernel, gamma = gamma, C = c)
            scores = cross_val_score(svm, svm_train, y_train, cv=3, scoring="accuracy")
            if np.mean(scores)> accuracy:
                accuracy, g, C = np.mean(scores), gamma, c
    return  accuracy, g , C

## Linear Kernel

In [6]:
linear_accuracy, g, C = cv("linear")
linear = SVC(kernel = "linear", gamma = g, C = C).fit(svm_train, y_train)
linear_pred = linear.predict(svm_test)
print(f"Training Accuracy: {linear_accuracy}")
print(f"Testing Accuracy: {accuracy_score(y_test, linear_pred)}")

Training Accuracy: 0.5888561675925307
Testing Accuracy: 0.6441351888667992


## RBF Kernel

In [7]:
rbf_accuracy , g, C = cv("linear")
rbf = SVC(kernel = "rbf", gamma = g, C = C).fit(svm_train, y_train)
rbf_pred = rbf.predict(svm_test)
print(f"Training Accuracy: {rbf_accuracy}")
print(f"Testing Accuracy: {accuracy_score(y_test, rbf_pred)}")

Training Accuracy: 0.5888561675925307
Testing Accuracy: 0.5049701789264414


## Poly Kernel

In [8]:
poly_accuracy, g, C = cv("poly")
poly = SVC(kernel = "poly", degree = 2,gamma = g, C = C).fit(svm_train, y_train)
poly_pred = poly.predict(svm_test)
print(f"Training Accuracy: {poly_accuracy}")
print(f"Testing Accuracy: {accuracy_score(y_test, poly_pred)}")

Training Accuracy: 0.6361458774884917
Testing Accuracy: 0.6660039761431411


Each kernel transforms the data into a higher dimensional space to make it easier to seperate out the data into classes. With a linear kernel, it means being able to slice through the data with a hyperplane, an RBF kernel creates different peaks from each centroid and the second degree poly kernel defines a hyperplane using the polynomial function. These kernels become better classifiers when the split in data corresponds to the kernel's shape.  

The training accuracy for linear and RBF kernel are similar, but linear does much better than RBF in testing accuracy. It shows that linear is more generalisable with less variance, making it more useful than the RBF kernel which performs much worse on unseen data. This possibly means that the RBF was overfit to the training data which is plausible considering that the RBF is more flexible than the linear and poly kernel. It uses multiple centroids to approximate a function for the data which even though reduces bias, increases its variance. On the other hand, the poly kernel outperforms the linear and RBF kernel in both training and testing accuracy which out of the three kernels makes it the best choice for classifying these images in SVM.

# Deep Learning (VGG16)

## Loss Function
I chose binary cross entropy because it is made for binary classification,increasing the penalty to the function the more wrong predictions made according to a log scale. Compared to a linear loss function, the more confident the algorithm was in a wrong prediction, the more it is penalised compared to only have a small confidence in a wrong prediction. This allows the loss function to converge at a minimum faster.

In [24]:
images = preprocess_input(X_train)

model = VGG16(weights="imagenet", include_top = False, input_shape=(64,64,3))

# Freeze weights
for layer in model.layers:
    layer.trainable = False

# Set classifier layer
x = Flatten(name='flatten')(model.layers[-2].output)
output = Dense(1, activation = "sigmoid", name = "prediction")(x)

# Define new model
model = Model(inputs=model.inputs, outputs=output)
model.compile(loss = "binary_crossentropy", metrics = ["accuracy"])

In [27]:
# Train model with images
model.fit(images, y_train.reshape((-1,1)), batch_size = 10, epochs = 10)

Epoch 1/10
201/201 [==============================] - 32s 160ms/step - loss: 0.1181 - accuracy: 0.9856
Epoch 2/10
201/201 [==============================] - 38s 190ms/step - loss: 0.1545 - accuracy: 0.9811
Epoch 3/10
201/201 [==============================] - 38s 191ms/step - loss: 0.0617 - accuracy: 0.9890
Epoch 4/10
201/201 [==============================] - 38s 191ms/step - loss: 0.1067 - accuracy: 0.9841
Epoch 5/10
201/201 [==============================] - 39s 193ms/step - loss: 0.0814 - accuracy: 0.9866
Epoch 6/10
201/201 [==============================] - 39s 195ms/step - loss: 0.0725 - accuracy: 0.9930
Epoch 7/10
201/201 [==============================] - 38s 190ms/step - loss: 0.0244 - accuracy: 0.9965
Epoch 8/10
201/201 [==============================] - 40s 198ms/step - loss: 0.0952 - accuracy: 0.9895
Epoch 9/10
201/201 [==============================] - 39s 192ms/step - loss: 0.0879 - accuracy: 0.9871
Epoch 10/10
201/201 [==============================] - 40s 197ms/step - l

In [28]:
VGG16_pred = model.predict(X_test)
print(f'Testing Accuracy: {(accuracy_score(y_test, np.round(VGG16_pred)))}')

Testing Accuracy: 0.5904572564612326


# SVM vs Deep Neural Networks

The classification by the neural network clearly outperforms SVM in training accuracy since it makes use of images from ImageNet to help identify salient features of images. However, it only gives a middling performance when it comes to testing accuracy, only more accurate than one out of the three SVM kernels.

The training accuracy is so high because it has a more flexible way of classifying images. The SVM pulls images into multi-dimensional space to classify them while being limited by our assumptions on the split in data whereas neural networks classify images based on specific features. This aspect is reflected in the sheer number of weights from ImageNet used to parse through the images.

A neural network is the a preferable method on a time constraint because it's training time is significantly shorter since the original weights from ImageNet don't need to be retrained while providing valuable information. Compared to SVM where it needs to be trained and cross-validated across two parameters, the neural network is more efficient. However, if you value accuracy and have the time to wait for it to train, the 5% push that the poly and linear kernel give is worth the few extra hours of training time.